# Border Crossing Data Analysis for U.S. Government

The Bureau of Transportation Statistics (BTS) collects monthly data on inbound crossings at the U.S.-Canada and U.S.-Mexico borders, covering trucks, trains, containers, buses, personal vehicles, passengers, and pedestrians. This data, collected by U.S. Customs and Border Protection (CBP), reflects the number of vehicles, containers, passengers, or pedestrians entering the United States at various ports.

**Mission:** As analysts, our goal is to extract actionable insights from this data to inform policy and resource allocation. Our findings will help optimize trade facilitation, enhance security measures, and foster international cooperation, aligning with the U.S. government's directive to improve border management and resource efficiency.

Here is the link to dataset: [CSV](https://docs.google.com/spreadsheets/d/1h0xr09DFPIRMr_mcnwuV63UveQ3sGHi-VGOlNBfjEBA/edit#gid=381927494)

In [44]:
# import data from csv file
import pandas as pd
import numpy as np

border_data = pd.read_csv('Border_Crossing_Entry_Data.csv')

# validate data 
border_data.sample(5)

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
262216,Port Angeles,Washington,3007,US-Canada Border,May 2005,Trucks,155,48.122,-123.431,POINT (-123.430694 48.121858)
66284,Lancaster,Minnesota,3430,US-Canada Border,May 2015,Truck Containers Loaded,55,49.000,-96.801,POINT (-96.800527 49.000194)
314944,Fort Fairfield,Maine,107,US-Canada Border,Jul 2001,Pedestrians,3,46.765,-67.789,POINT (-67.789471 46.765323)
255214,Eagle Pass,Texas,2303,US-Mexico Border,Feb 2004,Bus Passengers,1406,28.705,-100.512,POINT (-100.511977 28.705483)
268949,Maida,North Dakota,3416,US-Canada Border,Jun 2004,Trains,0,49.000,-98.365,POINT (-98.36525 49.00025)


In [45]:
# check for missing values
border_data.isna().sum()

Port Name    0
State        0
Port Code    0
Border       0
Date         0
Measure      0
Value        0
Latitude     0
Longitude    0
Point        0
dtype: int64

In [18]:
# convert date to datetime
border_data['Date'] = pd.to_datetime(border_data['Date'], format='%b %Y')

# create month and year columns
border_data['Month'] = border_data['Date'].dt.month_name()
border_data['Year'] = border_data['Date'].dt.year

In [35]:
# create a new column for the sum of the number of people crossing the border
border_data['Measure'].value_counts()


Measure
Personal Vehicles              36608
Personal Vehicle Passengers    36582
Trucks                         35617
Truck Containers Empty         35455
Truck Containers Loaded        35010
Pedestrians                    32019
Buses                          31150
Bus Passengers                 31132
Trains                         29506
Rail Containers Empty          29485
Rail Containers Loaded         29399
Train Passengers               29080
Name: count, dtype: int64

In [36]:
# create a new column for the sum of the number of people crossing the border in different type in 2024
df = border_data[(border_data['Year'] == 2024) & (border_data['Border'] == "US-Canada Border")]
df.groupby(["Month","Measure"])['Value'].sum()

Month     Measure                    
February  Bus Passengers                   36595
          Buses                             2257
          Pedestrians                       7841
          Personal Vehicle Passengers    2673872
          Personal Vehicles              1429367
          Rail Containers Empty            71961
          Rail Containers Loaded          138757
          Train Passengers                  6245
          Trains                            1796
          Truck Containers Empty          106630
          Truck Containers Loaded         629541
          Trucks                          448552
January   Bus Passengers                   30751
          Buses                             2443
          Pedestrians                      11942
          Personal Vehicle Passengers    2454742
          Personal Vehicles              1327146
          Rail Containers Empty            63934
          Rail Containers Loaded          128848
          Train Passengers     

In [40]:
# count the number of port 
border_data['Port Code'].nunique()

117

In [42]:
# count the number of port in each state
border_data.groupby(["State"])['Port Code'].nunique()

State
Alaska           4
Arizona          6
California       7
Idaho            2
Maine           13
Michigan         4
Minnesota        8
Montana         13
New Mexico       2
New York         7
North Dakota    18
Texas           13
Vermont          5
Washington      15
Name: Port Code, dtype: int64

In [47]:
# calculate the ratio of truck containers empty and loaded to the total number of trucks
truck = border_data.groupby(['Measure', 'Border', 'Port Name'])['Value'].sum().unstack('Measure')
truck['Ratio'] = (truck['Truck Containers Empty'] + truck['Truck Containers Loaded'])/truck['Trucks']
truck.sort_values(by = 'Ratio', ascending= False)

Measure                               Bus Passengers      Buses  Pedestrians  \
Border           Port Name                                                     
US-Canada Border Ketchikan                    2499.0      147.0      27035.0   
                 Bar Harbor                   9151.0      477.0      52397.0   
US-Mexico Border Tornillo                        0.0        0.0     998991.0   
US-Canada Border Dalton Cache                35193.0     4730.0      11980.0   
                 Vanceboro                    3739.0      430.0      12919.0   
...                                              ...        ...          ...   
                 Algonac                         NaN        NaN          NaN   
                 Cape Vincent                    0.0        0.0          0.0   
US-Mexico Border Boquillas                       0.0        0.0      98603.0   
                 Cross Border Xpress             NaN        NaN   10634357.0   
                 San Ysidro               16161405.0  1928798.0  210016547.0   

Measure                               Personal Vehicle Passengers  \
Border           Port Name                                          
US-Canada Border Ketchikan                                90987.0   
                 Bar Harbor                              162066.0   
US-Mexico Border Tornillo                              31716339.0   
US-Canada Border Dalton Cache                            993265.0   
                 Vanceboro                              2165348.0   
...                                                           ...   
                 Algonac                                 141070.0   
                 Cape Vincent                            252603.0   
US-Mexico Border Boquillas                                   56.0   
                 Cross Border Xpress                          NaN   
                 San Ysidro                           736182123.0   

Measure                               Personal Vehicles  \
Border           Port Name                                
US-Canada Border Ketchikan                      72147.0   
                 Bar Harbor                     71803.0   
US-Mexico Border Tornillo                    13931275.0   
US-Canada Border Dalton Cache                  466360.0   
                 Vanceboro                    1246092.0   
...                                                 ...   
                 Algonac                       176429.0   
                 Cape Vincent                  101508.0   
US-Mexico Border Boquillas                          0.0   
                 Cross Border Xpress                NaN   
                 San Ysidro                 394096855.0   

Measure                               Rail Containers Empty  \
Border           Port Name                                    
US-Canada Border Ketchikan                              0.0   
                 Bar Harbor                             0.0   
US-Mexico Border Tornillo                               0.0   
US-Canada Border Dalton Cache                           0.0   
                 Vanceboro                         264499.0   
...                                                     ...   
                 Algonac                                NaN   
                 Cape Vincent                           0.0   
US-Mexico Border Boquillas                              0.0   
                 Cross Border Xpress                    NaN   
                 San Ysidro                             0.0   

Measure                               Rail Containers Loaded  \
Border           Port Name                                     
US-Canada Border Ketchikan                               0.0   
                 Bar Harbor                              0.0   
US-Mexico Border Tornillo                                0.0   
US-Canada Border Dalton Cache                            0.0   
                 Vanceboro                          358944.0   
...                      